In [183]:
import pandas as pd
import numpy as np
import requests
import json
from pandas import json_normalize

from sklearn.cluster import KMeans
import folium

import matplotlib.cm as cm
import matplotlib.colors as colors


## We need lxml, html5 and beautifulsoup4 for web scraping

In [2]:
!pip install lxml html5lib beautifulsoup4

You should consider upgrading via the 'f:\anaconda\python.exe -m pip install --upgrade pip' command.


## Scrape the wikipedia webpage using pandas
###            We can use the read_html(url) function to get webpage contents

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
list_web=pd.read_html(url)

## Get the dataframe required dataframe from a list of dataframes
#### read_html(url) returns a list of all the tables from the webpage. We require the first table from the webpage. So we will be using list_web[0]  for our dataframe


In [4]:
df_web=list_web[0]
df_web.head(3)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods


## Remove all the rows containing 'Not assigned' in its 'Borough' column

In [5]:
df_web=df_web[df_web['Borough']!='Not assigned']
df_web.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Since we deleted a few rows, we will be needing to reset the index

In [6]:
df_web.reset_index(inplace=True)
df_web.drop('index',axis=1,inplace=True)

F:\anaconda\lib\site-packages\pandas\core\frame.py:4160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


## A final peek of our dataframe

In [7]:
df_web

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## No of rows and columns in our dataframe

In [8]:
df_web.shape

(103, 3)

In [9]:
url2='https://cocl.us/Geospatial_data'
df_cord=pd.read_csv(url2)
df_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df_final=pd.merge(df_web,df_cord,on='Postal Code')

In [11]:
df_final

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [12]:
temp=['Toronto' in each for each in df_final['Borough']]
df_toronto=df_final[temp]
df_toronto.head()
df_toronto.reset_index(drop=True,inplace=True)
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [13]:
a=df_toronto.loc[0,'Neighbourhood']
a=a.split(',')[0]
a
N_LATITUDE=df_toronto.loc[0,'Latitude']
N_LONGITUDE=df_toronto.loc[0,'Longitude']
type(N_LATITUDE),N_LATITUDE,N_LONGITUDE

(numpy.float64, 43.6542599, -79.3606359)

In [17]:
# CLIENT_ID = 'S01X3E5GTFLE31LD3QX2LNR2TSY0XIIE5DYXOUOR1OMEHXBH' # your Foursquare ID
# CLIENT_SECRET = 'B2QL1LH3NAM2LG0SQTXDJSY1EUOG4REZRT2TZUKA1ZNGZX1T' # your Foursquare Secret
CLIENT_ID='ZHN1PY5PXKH2V2NTF0VTQSXFFWO4E03EMBFVCZG0VLSGSUIO'
CLIENT_SECRET='Q15C4UOQLL5JV1FEW3E1OV0ODDMD24IGB1AOM35B4OZBSSTV'

VERSION = '20211010' # Foursquare API version
RADIUS=500
LIMIT=100

url3='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'
url4='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    N_LATITUDE,
    N_LONGITUDE,
    RADIUS,
    LIMIT
)

In [18]:
b=requests.get(url4).json()
# b=b['response']['groups'][0]['items']
# b=json_normalize(b)
# req_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
# b=b[req_columns]
b

{'meta': {'code': 200, 'requestId': '5fef06278028c13d49d16e12'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

In [33]:
c=json_normalize(b['response']['groups'][0]['items'])
c

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,...,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id,venue.location.neighborhood
0,e-0-54ea41ad498e9a11e9e13308-0,0,"[{'summary': 'This spot is popular', 'type': '...",54ea41ad498e9a11e9e13308,Roselle Desserts,362 King St E,Trinity St,43.653447,-79.362017,"[{'label': 'display', 'lat': 43.65344672305267...",...,CA,Toronto,ON,Canada,"[362 King St E (Trinity St), Toronto ON M5A 1K...","[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",0,[],NaN,NaN
1,e-0-53b8466a498e83df908c3f21-1,0,"[{'summary': 'This spot is popular', 'type': '...",53b8466a498e83df908c3f21,Tandem Coffee,368 King St E,at Trinity St,43.653559,-79.361809,"[{'label': 'display', 'lat': 43.65355870959944...",...,CA,Toronto,ON,Canada,"[368 King St E (at Trinity St), Toronto ON, Ca...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN
2,e-0-574c229e498ebb5c6b257902-2,0,"[{'summary': 'This spot is popular', 'type': '...",574c229e498ebb5c6b257902,Cooper Koo Family YMCA,461 Cherry St,NaN,43.653249,-79.358008,"[{'label': 'display', 'lat': 43.65324910177244...",...,CA,Toronto,ON,Canada,"[461 Cherry St, Toronto ON M5A 0H7, Canada]","[{'id': '52e81612bcbc57f1066b7a37', 'name': 'D...",0,[],NaN,NaN
3,e-0-4ae5b91ff964a520a6a121e3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4ae5b91ff964a520a6a121e3,Morning Glory Cafe,457 King St. E,Gilead Place,43.653947,-79.361149,"[{'label': 'display', 'lat': 43.65394694263529...",...,CA,Toronto,ON,Canada,"[457 King St. E (Gilead Place), Toronto ON M5A...","[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",0,[],39686393,NaN
4,e-0-50760559e4b0e8c7babe2497-4,0,"[{'summary': 'This spot is popular', 'type': '...",50760559e4b0e8c7babe2497,Body Blitz Spa East,497 King Street East,btwn Sackville St and Sumach St,43.654735,-79.359874,"[{'label': 'display', 'lat': 43.65473505045365...",...,CA,Toronto,ON,Canada,[497 King Street East (btwn Sackville St and S...,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",0,[],NaN,NaN
5,e-0-5612b1cc498e3dd742af0dc8-5,0,"[{'summary': 'This spot is popular', 'type': '...",5612b1cc498e3dd742af0dc8,Impact Kitchen,573 King St E,at St Lawrence St,43.656369,-79.356980,"[{'label': 'display', 'lat': 43.65636850543279...",...,CA,Toronto,ON,Canada,"[573 King St E (at St Lawrence St), Toronto ON...","[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",0,[],NaN,NaN
6,e-0-4e8b7fa1cc2112f67517660a-6,0,"[{'summary': 'This spot is popular', 'type': '...",4e8b7fa1cc2112f67517660a,The Extension Room,30 Eastern Ave,Sackville St.,43.653313,-79.359725,"[{'label': 'display', 'lat': 43.65331304337331...",...,CA,Toronto,ON,Canada,"[30 Eastern Ave (Sackville St.), Toronto ON, C...","[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",0,[],NaN,NaN
7,e-0-51ccc048498ec7792efc955e-7,0,"[{'summary': 'This spot is popular', 'type': '...",51ccc048498ec7792efc955e,Corktown Common,NaN,NaN,43.655618,-79.356211,"[{'label': 'display', 'lat': 43.65561779974973...",...,CA,NaN,NaN,Canada,[Canada],"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN,NaN
8,e-0-4ad4c05ef964a520bff620e3-8,0,"[{'summary': 'This spot is popular', 'type': '...",4ad4c05ef964a520bff620e3,The Distillery Historic District,"btwn Front, Cherry, Gardiner & Parliament",NaN,43.650244,-79.359323,"[{'label': 'display', 'lat': 43.65024435658077...",...,CA,Toronto,ON,Canada,"[btwn Front, Cherry, Gardiner & Parliament, To...","[{'id': '4deefb944765f83613cdba6e', 'name': 'H...",0,[],NaN,NaN
9,e-0-4b0978e1f964a520cd1723e3-9,0,"[{'summary': 'This spot is popular', 'type': '...",4b0978e1f964a520cd1723e3,SOMA chocolatemaker,"55 Mill Street, Unit #48",The Distillery District,43.650622,-79.358127,"[{'label': 'display', 'lat': 43.65062222570758...",...,CA

In [37]:
temp=c.loc[1,'venue.categories']
temp[0]['name']

'Coffee Shop'

In [49]:
# CLIENT_ID = 'S01X3E5GTFLE31LD3QX2LNR2TSY0XIIE5DYXOUOR1OMEHXBH' # your Foursquare ID
# CLIENT_SECRET = 'B2QL1LH3NAM2LG0SQTXDJSY1EUOG4REZRT2TZUKA1ZNGZX1T' # your Foursquare Secret
CLIENT_ID='ZHN1PY5PXKH2V2NTF0VTQSXFFWO4E03EMBFVCZG0VLSGSUIO'
CLIENT_SECRET='Q15C4UOQLL5JV1FEW3E1OV0ODDMD24IGB1AOM35B4OZBSSTV'

VERSION = '20211010' # Foursquare API version
RADIUS=500
LIMIT=100
url3='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'
url4='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    N_LATITUDE,
    N_LONGITUDE,
    RADIUS,
    LIMIT
)


In [41]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [151]:
hood=df_toronto['Neighbourhood']
lat=df_toronto['Latitude']
lon=df_toronto['Longitude']

len(hood),type(hood)

# df_venues=pd.DataFrame()
df_venues=pd.DataFrame(columns=['Neighbourhood','Neighbourhood Latitude','Neighbourhood Longitude','venue.name','venue.categories','venue.location.lat',\
                                'venue.location.lng'])


for i in range(len(hood)):
    N_LATITUDE=lat[i]
    N_LONGITUDE=lon[i]
    r=requests.get('https://api.foursquare.com/v2/venues/explore?\
    &client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat[i],
    lon[i],
    500,
    100
    )).json()
    df_temp=json_normalize(r['response']['groups'][0]['items'])
    req_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
    df_temp=df_temp[req_columns]
    df_temp['venue.categories']=df_temp.apply(get_category_type,axis=1)
    df_temp[['Neighbourhood','Neighbourhood Latitude','Neighbourhood Longitude']]=hood[i],lat[i],lon[i]
    df_venues=df_venues.append(df_temp)
    print(df_venues.shape)



(46, 7)
(77, 7)
(177, 7)
(255, 7)
(261, 7)
(319, 7)
(384, 7)
(400, 7)
(497, 7)
(515, 7)
(615, 7)
(657, 7)
(700, 7)
(800, 7)
(822, 7)
(840, 7)
(940, 7)
(977, 7)
(981, 7)
(985, 7)
(994, 7)
(999, 7)
(1023, 7)
(1042, 7)
(1061, 7)
(1075, 7)
(1113, 7)
(1147, 7)
(1182, 7)
(1186, 7)
(1249, 7)
(1263, 7)
(1279, 7)
(1283, 7)
(1382, 7)
(1428, 7)
(1528, 7)
(1607, 7)
(1624, 7)


In [152]:
# type(df_venues)
# df_venues=pd.DataFrame(df_venues)
# df_venues.iloc[1500,:]
len(df_venues['venue.categories'].unique())

235

In [153]:
df_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,Bakery,43.653447,-79.362017
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,Spa,43.654735,-79.359874
...,...,...,...,...,...,...,...
12,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Jonathan Ashbridge Park,Park,43.664702,-79.319898
13,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Toronto Yoga Mamas,Yoga Studio,43.664824,-79.324335
14,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Stop #03049,Light Rail Station,43.664470,-79.325145
15,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Greenwood Cigar & Variety,Smoke Shop,43.664538,-79.325379


In [154]:
a=df_venues.groupby('Neighbourhood').count()
a.head()

,Neighbourhood Latitude,Neighbourhood Longitude,venue.name,venue.categories,venue.location.lat,venue.location.lng
Neighbourhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,65,65,65,65,65,65


In [155]:
df_onehot=pd.get_dummies(df_venues[['venue.categories']],prefix="",prefix_sep="")
df_onehot['Neighbourhood']=df_venues['Neighbourhood']
order_columns=[df_onehot.columns[-1]]+list(df_onehot.columns[:-1])
df_onehot=df_onehot[order_columns]
df_onehot

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
14,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
df_toronto_mean=df_onehot.groupby('Neighbourhood').mean().reset_index()
# df_toronto_mean.reset_index(inplace=True)
df_toronto_mean

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.015385,0.000000,0.000000,0.015385,0.000000,0.015385
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.012658,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012658,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.012658,0.000000,0.000000,0.025316
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.026316,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [200]:
df_toronto_mean_2=df_toronto_mean.drop('Neighbourhood',axis=1)
n_clusters=3
kmeans=KMeans(n_clusters,random_state=0).fit(df_toronto_mean_2)
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])

In [201]:
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])

In [166]:
# CLIENT_ID='ZHN1PY5PXKH2V2NTF0VTQSXFFWO4E03EMBFVCZG0VLSGSUIO'
# CLIENT_SECRET='Q15C4UOQLL5JV1FEW3E1OV0ODDMD24IGB1AOM35B4OZBSSTV'

# VERSION = '20211010' # Foursquare API version
# RADIUS=500
# LIMIT=100
# url3='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'
# url4='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID,
#     CLIENT_SECRET,
#     VERSION,
#     N_LATITUDE,
#     N_LONGITUDE,
#     RADIUS,
#     LIMIT
# )
 
# temp=pd.DataFrame()

# for i in range(1):#len(hood)):
#     N_LATITUDE=lat[i]
#     N_LONGITUDE=lon[i]
#     r=requests.get(url4).json()
#     df_temp=json_normalize(r['response']['groups'][0]['items'])
#     req_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
#     df_temp=df_temp[req_columns]
#     df_temp['venue.categories']=df_temp.apply(get_category_type,axis=1)
#     df_temp[['Neighbourhood','Neighbourhood Latitude','Neighbourhood Longitude']]=hood[i],lat[i],lon[i]
#     temp=temp.append(df_temp)
#     print(temp.shape)

In [168]:
# a=df_venues.loc[:,'venue.categories']
# a[:55]

In [169]:
# hood=df_toronto['Neighbourhood']
# lat=df_toronto['Latitude']
# lon=df_toronto['Longitude']

# len(hood),type(hood)

# e=pd.DataFrame()
# # df_venues=pd.DataFrame(columns=['Neighbourhood','Neighbourhood Latitude','Neighbourhood Longitude','venue.name','venue.categories','venue.location.lat',\
# #                                 'venue.location.lng'])


# for i in range(len(hood)):
#     print(i)
#     N_LATITUDE=lat[i]
#     N_LONGITUDE=lon[i]
#     r=requests.get('https://api.foursquare.com/v2/venues/explore?\
#     &client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID,
#     CLIENT_SECRET,
#     VERSION,
#     lat[i],
#     lon[i],
#     500,
#     100
#     )).json()
#     df_temp=json_normalize(r['response']['groups'][0]['items'])
# #     req_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
# #     df_temp=df_temp[req_columns]
# #     df_temp['venue.categories']=df_temp.apply(get_category_type,axis=1)
# #     df_temp[['Neighbourhood','Neighbourhood Latitude','Neighbourhood Longitude']]=hood[i],lat[i],lon[i]
#     e=e.append(df_temp)
#     print(e.shape)


In [170]:
# for i in range(len(hood)):
#     print(i)
#     print(hood[i])
#     print(lat[i])
#     print(lon[i])

In [171]:
# f=e[['venue.name','venue.categories','venue.location.lat','venue.location.lng']]
# f

In [172]:
# # g=f[f['venue.name']=='The Sidekick']
# # g
# h=f['venue.name']=='Roselle Desserts'
# h.sum()

In [174]:
# f.head(25)

In [176]:
# f.groupby('venue.name').count()

In [202]:
temp=df_toronto.copy()
temp.insert(0,'Cluster Labels', kmeans.labels_)
temp.head()

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,1,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,1,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [203]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [204]:
# create map
latitude=43.6532
longitude=-79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(n_clusters)
ys = [i + x + (i*x)**2 for i in range(n_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(temp['Latitude'],temp['Longitude'], temp['Neighbourhood'], temp['Cluster Labels']):
    label = folium.Popup(str(poi) + '(Cluster ' + str(cluster)+')', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters